# Purpose
The purpose of this notebook is to apply post-hoc data fixes to results data. Any modification to the data must be retained in this notebook. No modification to performance metrics will be made. Only modifications to independent variables, or additions of new performance metrics will be performed.

In [1]:
import pandas as pd
import os
from tqdm import tqdm
from NlSqlBenchmark.snails.util import sqlite_db_util
from util.load_subsets_from_results import load_subsets_from_results
from NlSqlEvaluator.NlSqlPromptBuilder import NlSqlPromptBuilder
from NlSqlBenchmark.SchemaObjects import Schema, SchemaTable, TableColumn, ForeignKey
from NlSqlBenchmark.BenchmarkQuestion import BenchmarkQuestion
from NlSqlBenchmark.NlSqlBenchmarkFactory import NlSqlBenchmarkFactory
from SchemaSubsetter.Skalpel.LLM import LLM

NlSqlBenchmark.NlSqlBenchmarkFactory


/data/kluoma/skalpel-subsetting-evaluation/venv/lib/python3.10/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
/data/kluoma/skalpel-subsetting-evaluation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Only run this once!
cwd = os.getcwd()
cwd = os.path.dirname(cwd)
os.chdir(cwd)

### Add prompt token counts to already-completed CHESS subsetting runs

This traverses the CHESS logs and counts the tokens in the prompts used to generate subsets. It saves the counts to the results token_count column for each generated subset.

##### SNAILS

In [ ]:
from SchemaSubsetter.ChessSubsetter import ChessSubsetter
results_file = "./subsetting_results/archive/subsetting-chess-snails-Native-gpt4o.xlsx"
results_df = pd.read_excel(results_file)
for row in tqdm(results_df.itertuples(), total=results_df.shape[0]):
    if row.prompt_tokens == 0:
        token_counts, total_tokens = ChessSubsetter.get_token_counts_from_log(row.database, row.question_number)
    else:
        total_tokens = row.prompt_tokens
    results_df.at[row.Index, "prompt_tokens"] = total_tokens

100%|██████████| 503/503 [1:11:45<00:00,  8.56s/it]


In [17]:
results_df.to_excel("./subsetting_results/subsetting-chess-snails-Native-gpt4o.xlsx", index=False)

##### BIRD

In [4]:
from SchemaSubsetter.ChessSubsetter import ChessSubsetter
results_file = "./subsetting_results/archive/subsetting-chess-bird-Native-gpt4o.xlsx"
results_df = pd.read_excel(results_file)
for row in tqdm(results_df.itertuples(), total=results_df.shape[0]):
    if row.prompt_tokens == 0:
        token_counts, total_tokens = ChessSubsetter.get_token_counts_from_log(row.database, row.question_number)
    else:
        total_tokens = row.prompt_tokens
    results_df.at[row.Index, "prompt_tokens"] = total_tokens

NlSqlBenchmark.NlSqlBenchmarkFactory


c:\Users\kyle.luoma\Research Projects\SKALPEL-subsetting-evaluation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1534/1534 [35:21<00:00,  1.38s/it]


In [5]:
results_df.to_excel("./subsetting_results/subsetting-chess-bird-Native-gpt4o.xlsx", index=False)

### Index the Snails NYSED database to improve performance

In [3]:

sqlite_db_util.index_nysed_db(db_list_file="./benchmarks/snails/snails_sqlite/sqlite_dbinfo.json")

### Derive prompt token counts for subset schema prompts

In [3]:
nl_sql_files = [f for f in os.listdir("./nl_sql_results") if ".xlsx" in f]
subsetting_files = [f.replace("nltosql", "subsetting").split("-nlsqlmodel")[0] + ".xlsx" for f in nl_sql_files]
bm_fact = NlSqlBenchmarkFactory()
prompt_builder = NlSqlPromptBuilder()
llm = LLM()
for nlsql_f, subset_f in list(zip(nl_sql_files, subsetting_files)):
    nl_sql_prompt_tokens = []
    if "fullschema" in nlsql_f:
        continue
    bm_name = nlsql_f.split("-")[2]
    bm = bm_fact.build_benchmark(benchmark_name=bm_name)
    subsets = load_subsets_from_results("./subsetting_results/" + subset_f, bm)
    for subset, question in subsets[1]:
        quest_sub = question
        quest_sub.schema = subset
        prompt = prompt_builder.create_prompt(quest_sub)
        token_count = llm.get_prompt_token_count(prompt)
        break
    break

/data/kluoma/skalpel-subsetting-evaluation/src/util/StringObjectParser.py:58: UserWarning: Using eval can be dangerous. Make sure the input string is from a trusted source.
  warnings.warn("Using eval can be dangerous. Make sure the input string is from a trusted source.")


In [4]:
token_count

179